In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install setuptools==65.5.0
#@title <font size="5">← ឵឵<i>Upgrade FFmpeg to v5.0</font> { vertical-output: true }
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from google.colab import output, drive

from IPython.display import clear_output
import os, sys, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Cloning Repositories...", ty='twg')
!git clone https://github.com/XniceCraft/ffmpeg-colab.git
!chmod 755 ./ffmpeg-colab/install
textAn("Installing FFmpeg...", ty='twg')
!./ffmpeg-colab/install
clear_output()
print('Installation finished!')
!rm -fr /content/ffmpeg-colab
!ffmpeg -version

In [ ]:
# Install the necessary libraries
!pip install wheel==0.38.4
!pip install stable-baselines3[extra]
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install gym_super_mario_bros==7.3.0 nes_py
!pip install tensorboard

# **Import Libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import os
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
from gym.wrappers import RecordVideo
from time import time

class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Taken from https://stable-baselines3.readthedocs.io/en/master/guide/examples.html
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).
    :param check_freq:
    :param chk_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level.
    """
    def __init__(self, save_freq: int, check_freq: int, chk_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_freq = save_freq
        self.chk_dir = chk_dir
        self.save_path = os.path.join(chk_dir, 'models')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.save_freq == 0:
            if self.verbose > 0:
                print(f"Saving current model to {os.path.join(self.chk_dir, 'models')}")
            self.model.save(os.path.join(self.save_path, f'iter_{self.n_calls}'))

        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.chk_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print(f"Saving new best model to {os.path.join(self.chk_dir, 'best_model')}")
                  self.model.save(os.path.join(self.chk_dir, 'best_model'))

        return True

def startGameRand(env):    
    fin = True
    for step in range(100000): 
        if fin: 
            env.reset()
        state, reward, fin, info = env.step(env.action_space.sample())
        env.render()
    env.close()

def startGameModel(env, model):
    state = env.reset()
    while True: 
        action, _ = model.predict(state)
        state, _, _, _ = env.step(action)
        env.render()

def saveGameRand(env, len = 100000, dir = './videos/'):
    env = RecordVideo(env, dir + str(time()) + '/')
    fin = True
    for step in range(len): 
        if fin: 
            env.reset()
        state, reward, fin, info = env.step(env.action_space.sample())
    env.close()    

def saveGameModel(env, model, len = 100000, dir = './videos/'):
    env = RecordVideo(env, dir + str(time()) + '/')
    fin = True
    for step in range(len): 
        if fin: 
            state = env.reset()
        action, _ = model.predict(state)
        state, _, fin, _ = env.step(action)
    env.close()      

In [ ]:
# Import the game
import gym_super_mario_bros
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

# Import preprocessing wrappers
# Framestack allows us to stack multiple frames together
# GrayScaleObservation converts the frames to grayscale
# Performance is improved by reducing the size of the frames
from gym.wrappers import GrayScaleObservation, FrameStack
# Import the vectorized environment
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecMonitor
from matplotlib import pyplot as plt
# Import PPO for training
from stable_baselines3 import PPO
# Import PPO for training
from stable_baselines3 import DQN

# **Start the environment**

In [ ]:
# Generate the game environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

# Limit the action space to the SIMPLE_MOVEMENT
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Note: If some error occurs in training, run this cell again.
# Error is caused by dimensions of the observation space.

To better comprehend the training process, print the properties of the environment

In [ ]:
# Actions Mario can do 
SIMPLE_MOVEMENT
# Number of button combinations

In [ ]:
# Shape of the observation space
env.observation_space.shape

In [ ]:
# Number of button combinations
env.action_space

In [ ]:
# Do a random action
SIMPLE_MOVEMENT[env.action_space.sample()]

In [ ]:
# Properties of the frame that the game will be displayed on
env.observation_space

# **Preprocessing**

In [ ]:
# Grayscale reduces dimensionality
# and improve performance (fewer data to process)
env = GrayScaleObservation(env, keep_dim=True)
# Note: If 'MLPPolicy' is used, then keep_dim=False

# Vectorize the environment
env = DummyVecEnv([lambda: env])

# Stack frames
env = VecFrameStack(env, 4)
# Monitor your progress
env = VecMonitor(env, "/content/drive/MyDrive/Colab Notebooks/MarioData/PPO_altered_env/train/")

# Set the directories
CHECKPOINT_DIR = "/content/drive/MyDrive/Colab Notebooks/MarioData/PPO_altered_env/train/"
LOG_DIR = "/content/drive/MyDrive/Colab Notebooks/MarioData/PPO_altered_env/logs/"

# Create the callback: check every 1000 steps
callback = SaveOnBestTrainingRewardCallback(save_freq=100000, check_freq=1000, chk_dir=CHECKPOINT_DIR)

Another way of preprocessing

In [ ]:
# Grayscale reduces dimensionality
# and improve performance (fewer data to process)
env = GrayScaleObservation(env, keep_dim=False)
# Note: If 'MLPPolicy' is used, then keep_dim=False

# Vectorize the environment
env = DummyVecEnv([lambda: env])

# Stack frames
env = VecFrameStack(env, 2)
# Monitor your progress
env = VecMonitor(env, "/content/drive/MyDrive/Colab Notebooks/MarioData/DQN_withMLP/train/")

# Set the directories
CHECKPOINT_DIR = "/content/drive/MyDrive/Colab Notebooks/MarioData/DQN_withMLP/train/"
LOG_DIR = "/content/drive/MyDrive/Colab Notebooks/MarioData/DQN_withMLP/logs/"

# Create the callback: check every 1000 steps
callback = SaveOnBestTrainingRewardCallback(save_freq=100000, check_freq=1000, chk_dir=CHECKPOINT_DIR)

# **Training**

In [ ]:
# PPO Training 

# Create the model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.0001, n_steps=512, ent_coef=0.01, vf_coef=0.5)
# Start the training
model.learn(total_timesteps=1000000, log_interval=1, callback=callback)

Train another model with a different algorithm

In [ ]:
# DQN Training 

# Create the model
model = DQN('MlpPolicy', env, batch_size=192, verbose=1, learning_starts=10000, learning_rate=5e-3, exploration_fraction=0.1, exploration_initial_eps=1.0, exploration_final_eps=0.1, train_freq=8, buffer_size=10000, tensorboard_log= LOG_DIR )
# Start the training
model.learn(total_timesteps=1000000, log_interval=1, callback=callback)

# **Test the model**

In [ ]:
# Load the model
model = PPO.load(CHECKPOINT_DIR + "best_model", env=env)
# Test the model
state = env.reset()

while True:
    action, _states = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()


# **Resume training**

In [ ]:
# Resume a training 

# Load the most recent zip file
model_dqn = DQN.load("/content/drive/.shortcut-targets-by-id/1-3c5hKVTtHh3lW01y3paib-wraxEX6qO/DQN_default/train/models/iter_700000", tensorboard_log=LOG_DIR)
# Set the environment
model_dqn.set_env(env)
# Resume the training
model_dqn.learn(total_timesteps=1000001, callback=callback, log_interval=1, tb_log_name="DQN_1", reset_num_timesteps=False)

# Remark:
# As the training continues, naming of the zip files may cause error, overwriting the previous written files

# **Plot using TensorBoard**

In [ ]:
%cd drive/MyDrive/DQN_default

In [ ]:
%load_ext tensorboard

In [ ]:
%ls

In [ ]:
# There must be a file named "logs" within the current directory 
# that has the log files in it
%tensorboard --logdir logs

# Another method:
# 1. Download the log files to your local device
# 2. Open terminal
# 3. Activate an environment
# 4. Type " %tensorboard --logdir logs "
# 5. Copy the link to your browser to see the plots


**Trained models:**


1.   PPO_1 (learning rate = 0.000001)
2.   PPO_2 (learning rate = 0.00001)
3.   PPO_3 ('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.0001, n_steps=512, ent_coef=0.01, vf_coef=0.5)
4.   DQN_1 ('CnnPolicy', env, batch_size=192, verbose=1, learning_starts=10000, learning_rate=5e-3, exploration_fraction=0.1, exploration_initial_eps=1.0, exploration_final_eps=0.1, train_freq=8, buffer_size=10000, tensorboard_log= LOG_DIR )
5.   DQN_2 ('MlpPolicy', env, batch_size=192, verbose=1, learning_starts=10000, learning_rate=5e-3, exploration_fraction=0.1, exploration_initial_eps=1.0, exploration_final_eps=0.1, train_freq=8, buffer_size=10000, tensorboard_log= LOG_DIR )
6.   DQN_3('CnnPolicy', env, batch_size=256, verbose=1,learning_starts=10000 learning_rate=1e-4, exploration_fraction=0.2, exploration_initial_eps=1.0,exploration_final_eps=0.05, train_freq=16, buffer_size=50000, tensorboard_log=LOG_DIR)




